In [ ]:
import dash
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go

import app_layout as al
import mylib as my

import inference as infr

import pandas as pd
import numpy as np
import sys, os
import datetime
import pickle as pkl

In [ ]:
items = ['button1','time1', 'pred1', 'graph1', 'table1']
model = pkl.load(open("model.pkl","rb"))
trans = pkl.load(open("transform.pkl","rb"))

In [ ]:
app = dash.Dash()
app.layout =  al.app_layout(items)

In [ ]:
@app.callback(
    Output('time1','children'),
    Output('table1','data'),
    Output('pred1', 'children'),
    Output('graph1', 'figure'),
    Input('button1','n_clicks'),
    prevent_initial_call=False,
)
def fn(n_clicks): 
    
    tt = str(datetime.datetime.now())
    
    df = my.db_to_df_random(db_name='steel.db', table_name='test')
    pred, x_cols = infr.inference(trans, model, df)
    
    pred_df = pd.DataFrame(data=pred, columns=["pred"])
    my.df_to_db(df[x_cols], "operation.db", "input_x")
    my.df_to_db(pred_df, "operation.db","pred")
    
    print_df = my.db_to_df("operation.db","input_x")
    pred_history = my.db_to_df("operation.db","pred")

    fig=px.line(x=pred_history.index.tolist(), y=pd.to_numeric(pred_history['pred']))
    out=print_df.to_dict('records')
    
    return tt, out, pred, fig

In [ ]:
app.run_server(host="0.0.0.0", port=9101) 